In [1]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,17 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.3
H2O_cluster_version_age:,"14 days, 1 hour and 30 minutes"
H2O_cluster_name:,bonay
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,9 Gb
H2O_cluster_total_cores:,11
H2O_cluster_allowed_cores:,11
H2O_cluster_status:,"locked, healthy"


In [2]:
from h2o.automl import H2OAutoML

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression

In [4]:

# Cargar el dataset
file_path = 'dataset/itesal.csv'
data = pd.read_csv(file_path, quotechar='"', delimiter=";")

columns_to_drop = ['EMPRESA', 'NUMERO_FACTURA', 'NUMERO_ASIENTO_BORRADOR', 'TIPO_FACTURA', 'DIARIO', 'IMPORTE_COBRADO_FRA', 'CUENTA_CONTABLE','NUM_EFECTOS_COBRADOS', 'NUM_EFECTOS_PARCIAL', 'NUM_EFECTOS_IMPAGADO', 'NUM_EFECTOS_FUERA_PLAZO', 'NUM_EFECTOS_PDTE_EN_PLAZO']
data.drop(columns=columns_to_drop, inplace=True)

# Split the data into training and testing sets
train_data = data.sample(frac=0.8, random_state=0)
test_data = data.drop(train_data.index)
# load the data into an autogluon tabular dataset
label = 'CATEGORIA_FACTURA'

In [5]:
train = h2o.H2OFrame(train_data)
test = h2o.H2OFrame(test_data)
y = "CATEGORIA_FACTURA"
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()    
x = list(train.columns)
x.remove(y)

aml = H2OAutoML(max_models=20, seed=1, max_runtime_secs=600, sort_metric='mean_per_class_error', distribution='multinomial')
aml.train(x=x, y=y, training_frame=train)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
18:21:39.255: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_4_AutoML_1_20240625_182139


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    54                 270                         4.54414e+06            10           10           10            68            619           411.126

ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.010380164617771167
RMSE: 0.10188309289460723
LogLoss: 0.060455358145549205
Mean Per-Class Error: 0.03179964575479275
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
-2    -1    0    1      2      Error       Rate
----  ----  ---  -----  -----  ----------  ------------
126   0     0    13     2      0.106383    15 / 141
0     5520  0    0      14     0.00252982  14 / 5,534
0     0     573  17     4      0.0353535   21 / 594
0     0     0    18338  176    0.00950632  176 / 18,514
0     0     0    202    38454  0.00522558  202 / 38,656
126   5520  573  18570  38650  0.00674664  428 / 63,439

Top-5 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.993253
2    0.999953
3    1
4    1
5    1

ModelMetricsMultinomial: gbm
** Reported on cross-validation data. **

MSE: 0.08812652562372664
RMSE: 0.29686112177873114
LogLoss: 0.3147393720998169
Mean Per-Class Error: 0.37355044726914977
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
-2    -1    0    1      2      Error      Rate
----  ----  ---  -----  -----  ---------  --------------
2     0     0    54     85     0.985816   139 / 141
0     5396  28   23     87     0.0249368  138 / 5,534
0     31    258  203    102    0.565657   336 / 594
3     2     8    14293  4208   0.22799    4,221 / 18,514
1     4     3    2441   36207  0.0633537  2,449 / 38,656
6     5433  297  17014  40689  0.114803   7,283 / 63,439

Top-5 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.885197
2    0.991267
3    0.996658
4    0.999007
5    1

Cross-Validation Metrics Summary: 
                         mean       sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
-----------------------  ---------  ----------  ------------  ------------  ------------  ------------  ------------
accuracy                 0.885622   0.00331946  0.888871      0.883985      0.888793      0.881069      0.885394
aic                      nan        0           nan           nan           nan           nan           nan
auc                      nan        0           nan           nan           nan           nan           nan
err                      0.114378   0.00331946  0.111129      0.116015      0.111207      0.118931      0.114605
err_count                1451.2     42.1153     1410          1472          1411          1509          1454
loglikelihood            nan        0           nan           nan           nan           nan           nan
logloss                  0.321092   0.0072719   0.311677      0.329497      0.315815      0.322711      0.325758
max_per_class_error      0.985961   0.0192292   1             0.964286      0.9655

In [6]:
aml.leaderboard

model_id,mean_per_class_error,logloss,rmse,mse
GBM_4_AutoML_1_20240625_182139,0.37355,0.314739,0.296861,0.0881265
GBM_2_AutoML_1_20240625_182139,0.373924,0.318077,0.301319,0.0907931
GBM_3_AutoML_1_20240625_182139,0.373973,0.31398,0.299194,0.0895172
GBM_5_AutoML_1_20240625_182139,0.377381,0.325664,0.304844,0.0929301
GBM_1_AutoML_1_20240625_182139,0.378317,0.322348,0.301318,0.0907924
DRF_1_AutoML_1_20240625_182139,0.380491,0.398027,0.299834,0.0899003
XRT_1_AutoML_1_20240625_182139,0.424547,0.429862,0.359629,0.129333
GLM_1_AutoML_1_20240625_182139,0.717251,0.741305,0.506123,0.25616


In [8]:
perf = aml.leader.model_performance(test)

In [ ]:
aml.predict(test)

In [9]:

perf.show()

ModelMetricsMultinomial: gbm
** Reported on test data. **

MSE: 0.06034078465314379
RMSE: 0.24564361309251212
LogLoss: 0.23540161547356475
Mean Per-Class Error: 0.16776903330258136
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
-2    -1    0      1      2      Error       Rate
----  ----  -----  -----  -----  ----------  --------------
12    0     0      18     0      0.6         18 / 30
1     3005  1      5      5      0.00397746  12 / 3,017
1     0     11488  418    29     0.0375335   448 / 11,936
7     23    92     15219  1248   0.0825848   1,370 / 16,589
1     0     7      1576   12220  0.114749    1,584 / 13,804
22    3028  11588  17236  13502  0.0756347   3,432 / 45,376

Top-5 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.924365
2    0.990788
3    0.999515
4    0.99978
5    1

In [10]:
perf.confusion_matrix()

-2,-1,0,1,2,Error,Rate
12.0,0.0,0.0,18.0,0.0,0.6,18 / 30
1.0,3005.0,1.0,5.0,5.0,0.0039775,"12 / 3,017"
1.0,0.0,11488.0,418.0,29.0,0.0375335,"448 / 11,936"
7.0,23.0,92.0,15219.0,1248.0,0.0825848,"1,370 / 16,589"
1.0,0.0,7.0,1576.0,12220.0,0.1147493,"1,584 / 13,804"
22.0,3028.0,11588.0,17236.0,13502.0,0.0756347,"3,432 / 45,376"


In [7]:
#We need to get the predictions from the model: accuracy, precision, recall, f1-score
predictions = aml.predict(test)
predictions = predictions.as_data_frame()
predictions = predictions['predict'].values
test = test.as_data_frame()
test = test['CATEGORIA_FACTURA'].values


gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'NUMERO_SERIE' has levels not trained on: ["R73"]
  warnings.warn(w)
/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'FECHA_FACTURA' has levels not trained on: ["02/07/2022", "06/04/2023", "13/04/2022", "13/08/2019", "14/06/2018", "17/04/2018", "23/01/2018", "24/04/2020", "26/08/2021", "29/06/2023", "29/12/2020", "30/08/2022"]
  warnings.warn(w)
/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'FORMA_COBRO' has levels not trained on: ["CF63", "T60D"]
  warnings.warn(w)
/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'LIQUIDO_FACTURA' has levels not trained on: ["0,21", "0,4", "0,52", "0,56"

In [8]:
#show the results
print("Accuracy: ", accuracy_score(test, predictions))
print("Precision: ", precision_score(test, predictions, average='weighted'))
print("Recall: ", recall_score(test, predictions, average='weighted'))
print("F1-Score: ", f1_score(test, predictions, average='weighted'))
print(classification_report(test, predictions))
print(confusion_matrix(test, predictions))


Accuracy:  0.8909205548549811
Precision:  0.889644180954757
Recall:  0.8909205548549811
F1-Score:  0.8880108586188731
              precision    recall  f1-score   support

          -2       0.67      0.04      0.08        50
          -1       1.00      0.97      0.98      1365
           0       0.89      0.48      0.62       140
           1       0.86      0.79      0.82      4702
           2       0.89      0.94      0.92      9603

    accuracy                           0.89     15860
   macro avg       0.86      0.64      0.68     15860
weighted avg       0.89      0.89      0.89     15860

[[   2    0    0   15   33]
 [   0 1322    7    5   31]
 [   0    2   67   45   26]
 [   0    1    0 3699 1002]
 [   1    0    1  561 9040]]


In [11]:
perf.hit_ratio_table()

k,hit_ratio
1,0.9243653
2,0.9907880
3,0.9995151
4,0.9997796
5,0.9999999
